In [90]:
import tensorflow as tf
import re
import numpy as np

### Get the corpus

In [11]:
corpus = [
    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',
    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

### Create vocabulary

In [36]:
vocab = set(re.sub(r' +', ' ', ' '.join(corpus)).strip().split(' '))
vocab

{'Exceptionally',
 'I',
 'It',
 'Its',
 'Never',
 'The',
 'This',
 'Wonderful',
 'Wonderfully',
 'You',
 'a',
 'acting',
 'an',
 'and',
 'boring',
 'brillent',
 'brilliant',
 'did',
 'directed',
 'excellent',
 'executed',
 'fantastic',
 'good',
 'horrible',
 'is',
 'it',
 'like',
 'money',
 'move',
 'movie',
 'not',
 'of',
 'pathetic',
 'picture',
 'recommend',
 'series',
 'should',
 'such',
 'the',
 'very',
 'was',
 'waste',
 'watch',
 'watched',
 'will'}

In [35]:
word2id = {word:i for i, word in enumerate(vocab)}
word2id

{'good': 0,
 'brilliant': 1,
 'series': 2,
 'watched': 3,
 'should': 4,
 'was': 5,
 'and': 6,
 'such': 7,
 'will': 8,
 'Its': 9,
 'I': 10,
 'directed': 11,
 'an': 12,
 'picture': 13,
 'fantastic': 14,
 'brillent': 15,
 'watch': 16,
 'a': 17,
 'Wonderfully': 18,
 'The': 19,
 'the': 20,
 'movie': 21,
 'move': 22,
 'not': 23,
 'Exceptionally': 24,
 'money': 25,
 'it': 26,
 'waste': 27,
 'boring': 28,
 'It': 29,
 'recommend': 30,
 'is': 31,
 'Wonderful': 32,
 'acting': 33,
 'like': 34,
 'executed': 35,
 'very': 36,
 'of': 37,
 'Never': 38,
 'pathetic': 39,
 'horrible': 40,
 'excellent': 41,
 'You': 42,
 'did': 43,
 'This': 44}

In [21]:
id2word = list(vocab)
id2word

['good',
 'brilliant',
 'series',
 'watched',
 'should',
 'was',
 'and',
 'such',
 'will',
 'Its',
 'I',
 'directed',
 'an',
 'picture',
 'fantastic',
 'brillent',
 'watch',
 'a',
 'Wonderfully',
 'The',
 'the',
 'movie',
 'move',
 'not',
 'Exceptionally',
 'money',
 'it',
 'waste',
 'boring',
 'It',
 'recommend',
 'is',
 'Wonderful',
 'acting',
 'like',
 'executed',
 'very',
 'of',
 'Never',
 'pathetic',
 'horrible',
 'excellent',
 'You',
 'did',
 'This']

### one hot encoding for word

In [23]:
def ohe(word, word2id, dim=45):
    vec = [0]*dim
    vec[word2id[word]] = 1
    return vec

### Create training data

In [56]:
def create_training_data_from_a_sentence(words, window_size):
    ans = []
    lookup = (window_size-1)//2
    for i in range(len(words)):
        if (i-lookup) >= 0 and (i+lookup) < len(words):
            x = []
            j = i-lookup
            while j <= (i+lookup):
                if j != i:
                    x.append(words[j])
                j += 1
            ans.append([x, words[i]])
    return ans

window_size = 3 # keep it odd
training_data = [] # [[x1,x2],y1]
for i, line in enumerate(corpus):
    words = line.split(' ')
    training_data.extend(create_training_data_from_a_sentence(words, window_size))

training_data

[[['This', 'an'], 'is'],
 [['is', 'excellent'], 'an'],
 [['an', 'movie'], 'excellent'],
 [['The', 'was'], 'move'],
 [['move', 'fantastic'], 'was'],
 [['was', 'I'], 'fantastic'],
 [['fantastic', 'like'], 'I'],
 [['I', 'it'], 'like'],
 [['You', 'watch'], 'should'],
 [['should', 'it'], 'watch'],
 [['watch', 'is'], 'it'],
 [['it', 'brilliant'], 'is'],
 [['Wonderfully', 'and'], 'directed'],
 [['directed', 'executed'], 'and'],
 [['and', 'I'], 'executed'],
 [['executed', 'like'], 'I'],
 [['I', 'it'], 'like'],
 [['Its', 'fantastic'], 'a'],
 [['a', 'series'], 'fantastic'],
 [['Never', 'such'], 'watched'],
 [['watched', 'a'], 'such'],
 [['such', 'brillent'], 'a'],
 [['a', 'movie'], 'brillent'],
 [['It', 'a'], 'is'],
 [['is', 'Wonderful'], 'a'],
 [['a', 'movie'], 'Wonderful'],
 [['waste', 'money'], 'of'],
 [['It', 'very'], 'was'],
 [['was', 'boring'], 'very'],
 [['I', 'not'], 'did'],
 [['did', 'like'], 'not'],
 [['not', 'the'], 'like'],
 [['like', 'movie'], 'the'],
 [['The', 'was'], 'movie'],
 [[

In [72]:
from functools import reduce

X  = []
Y = []

for x, y in training_data:
    Y.append(ohe(y, word2id, 45))
    X.append(list(reduce(lambda u, v: [i|j for i, j in zip(u, v)], [ohe(word, word2id, 45) for word in x])))

In [73]:
len(X), len(Y), len(X[0]), len(Y[0])

(39, 39, 45, 45)

In [130]:
# sanity check of ith sample
i = 13
[[id2word[j] for j in range(len(X[i])) if X[i][j] == 1], [id2word[j] for j in range(len(Y[i])) if Y[i][j] == 1]]


[['directed', 'executed'], ['and']]

In [91]:
X_train = np.array(X)
Y_train = np.array(Y)
X_train, Y_train

(array([[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]))

In [92]:
X_train.shape, Y_train.shape

((39, 45), (39, 45))

### Neural network

In [118]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation='linear', input_shape=[45]),
  tf.keras.layers.Dense(45)
])

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=500)

Epoch 1/500
2/2 [==============================] - 0s 997us/step - loss: 3.8475 - accuracy: 0.0256
Epoch 2/500
2/2 [==============================] - 0s 997us/step - loss: 3.8391 - accuracy: 0.0256
Epoch 3/500
2/2 [==============================] - 0s 998us/step - loss: 3.8320 - accuracy: 0.0256
Epoch 4/500
2/2 [==============================] - 0s 987us/step - loss: 3.8251 - accuracy: 0.0513
Epoch 5/500
2/2 [==============================] - 0s 1ms/step - loss: 3.8184 - accuracy: 0.0513
Epoch 6/500
2/2 [==============================] - 0s 2ms/step - loss: 3.8119 - accuracy: 0.0513
Epoch 7/500
2/2 [==============================] - 0s 499us/step - loss: 3.8054 - accuracy: 0.0513
Epoch 8/500
2/2 [==============================] - 0s 996us/step - loss: 3.7991 - accuracy: 0.0769
Epoch 9/500
2/2 [==============================] - 0s 1ms/step - loss: 3.7928 - accuracy: 0.0769
Epoch 10/500
2/2 [==============================] - 0s 2ms/step - loss: 3.7866 - accuracy: 0.0769
Epoch 11/500
2/2 

2/2 [==============================] - 0s 1ms/step - loss: 3.2833 - accuracy: 0.4615
Epoch 84/500
2/2 [==============================] - 0s 1ms/step - loss: 3.2742 - accuracy: 0.4615
Epoch 85/500
2/2 [==============================] - 0s 996us/step - loss: 3.2652 - accuracy: 0.4615
Epoch 86/500
2/2 [==============================] - 0s 997us/step - loss: 3.2564 - accuracy: 0.4615
Epoch 87/500
2/2 [==============================] - 0s 1ms/step - loss: 3.2468 - accuracy: 0.4615
Epoch 88/500
2/2 [==============================] - 0s 1ms/step - loss: 3.2377 - accuracy: 0.4615
Epoch 89/500
2/2 [==============================] - 0s 998us/step - loss: 3.2284 - accuracy: 0.4615
Epoch 90/500
2/2 [==============================] - 0s 997us/step - loss: 3.2189 - accuracy: 0.4615
Epoch 91/500
2/2 [==============================] - 0s 1ms/step - loss: 3.2093 - accuracy: 0.4615
Epoch 92/500
2/2 [==============================] - 0s 998us/step - loss: 3.1997 - accuracy: 0.4615
Epoch 93/500
2/2 [=====

Epoch 165/500
2/2 [==============================] - 0s 1ms/step - loss: 2.3977 - accuracy: 0.5641
Epoch 166/500
2/2 [==============================] - 0s 997us/step - loss: 2.3866 - accuracy: 0.5641
Epoch 167/500
2/2 [==============================] - 0s 997us/step - loss: 2.3753 - accuracy: 0.5641
Epoch 168/500
2/2 [==============================] - 0s 2ms/step - loss: 2.3640 - accuracy: 0.5897
Epoch 169/500
2/2 [==============================] - 0s 996us/step - loss: 2.3531 - accuracy: 0.5897
Epoch 170/500
2/2 [==============================] - 0s 2ms/step - loss: 2.3419 - accuracy: 0.5897
Epoch 171/500
2/2 [==============================] - 0s 2ms/step - loss: 2.3307 - accuracy: 0.5897
Epoch 172/500
2/2 [==============================] - 0s 1ms/step - loss: 2.3197 - accuracy: 0.5897
Epoch 173/500
2/2 [==============================] - 0s 1ms/step - loss: 2.3084 - accuracy: 0.5641
Epoch 174/500
2/2 [==============================] - 0s 2ms/step - loss: 2.2978 - accuracy: 0.5641
Epoc

Epoch 247/500
2/2 [==============================] - 0s 1ms/step - loss: 1.5794 - accuracy: 0.6667
Epoch 248/500
2/2 [==============================] - 0s 499us/step - loss: 1.5708 - accuracy: 0.6667
Epoch 249/500
2/2 [==============================] - 0s 1ms/step - loss: 1.5624 - accuracy: 0.6667
Epoch 250/500
2/2 [==============================] - 0s 997us/step - loss: 1.5541 - accuracy: 0.6667
Epoch 251/500
2/2 [==============================] - 0s 998us/step - loss: 1.5457 - accuracy: 0.6667
Epoch 252/500
2/2 [==============================] - 0s 996us/step - loss: 1.5374 - accuracy: 0.6923
Epoch 253/500
2/2 [==============================] - 0s 997us/step - loss: 1.5291 - accuracy: 0.6923
Epoch 254/500
2/2 [==============================] - 0s 998us/step - loss: 1.5209 - accuracy: 0.7179
Epoch 255/500
2/2 [==============================] - 0s 1ms/step - loss: 1.5126 - accuracy: 0.7179
Epoch 256/500
2/2 [==============================] - 0s 499us/step - loss: 1.5045 - accuracy: 0.7

2/2 [==============================] - 0s 1ms/step - loss: 1.0236 - accuracy: 0.7949
Epoch 329/500
2/2 [==============================] - 0s 505us/step - loss: 1.0183 - accuracy: 0.7949
Epoch 330/500
2/2 [==============================] - 0s 498us/step - loss: 1.0129 - accuracy: 0.7949
Epoch 331/500
2/2 [==============================] - 0s 985us/step - loss: 1.0080 - accuracy: 0.7949
Epoch 332/500
2/2 [==============================] - 0s 1ms/step - loss: 1.0029 - accuracy: 0.7949
Epoch 333/500
2/2 [==============================] - 0s 1ms/step - loss: 0.9976 - accuracy: 0.7949
Epoch 334/500
2/2 [==============================] - 0s 998us/step - loss: 0.9926 - accuracy: 0.7949
Epoch 335/500
2/2 [==============================] - 0s 1ms/step - loss: 0.9876 - accuracy: 0.7949
Epoch 336/500
2/2 [==============================] - 0s 997us/step - loss: 0.9826 - accuracy: 0.7949
Epoch 337/500
2/2 [==============================] - 0s 998us/step - loss: 0.9777 - accuracy: 0.7949
Epoch 338/50

2/2 [==============================] - 0s 1ms/step - loss: 0.6844 - accuracy: 0.8718
Epoch 410/500
2/2 [==============================] - 0s 997us/step - loss: 0.6809 - accuracy: 0.8974
Epoch 411/500
2/2 [==============================] - 0s 998us/step - loss: 0.6778 - accuracy: 0.8974
Epoch 412/500
2/2 [==============================] - 0s 499us/step - loss: 0.6744 - accuracy: 0.8974
Epoch 413/500
2/2 [==============================] - 0s 2ms/step - loss: 0.6712 - accuracy: 0.8974
Epoch 414/500
2/2 [==============================] - 0s 998us/step - loss: 0.6680 - accuracy: 0.9231
Epoch 415/500
2/2 [==============================] - 0s 997us/step - loss: 0.6649 - accuracy: 0.9231
Epoch 416/500
2/2 [==============================] - 0s 996us/step - loss: 0.6618 - accuracy: 0.9231
Epoch 417/500
2/2 [==============================] - 0s 1ms/step - loss: 0.6587 - accuracy: 0.9231
Epoch 418/500
2/2 [==============================] - 0s 997us/step - loss: 0.6557 - accuracy: 0.9231
Epoch 419/

2/2 [==============================] - 0s 996us/step - loss: 0.4801 - accuracy: 0.9231
Epoch 491/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4779 - accuracy: 0.9231
Epoch 492/500
2/2 [==============================] - 0s 2ms/step - loss: 0.4760 - accuracy: 0.9231
Epoch 493/500
2/2 [==============================] - 0s 997us/step - loss: 0.4740 - accuracy: 0.9231
Epoch 494/500
2/2 [==============================] - 0s 997us/step - loss: 0.4722 - accuracy: 0.9231
Epoch 495/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4703 - accuracy: 0.9231
Epoch 496/500
2/2 [==============================] - 0s 1ms/step - loss: 0.4684 - accuracy: 0.9231
Epoch 497/500
2/2 [==============================] - 0s 997us/step - loss: 0.4667 - accuracy: 0.9231
Epoch 498/500
2/2 [==============================] - 0s 997us/step - loss: 0.4649 - accuracy: 0.9231
Epoch 499/500
2/2 [==============================] - 0s 998us/step - loss: 0.4632 - accuracy: 0.9231
Epoch 500/50

In [128]:
w, b = model.layers[0].get_weights()
w.shape, b.shape

((45, 10), (10,))

### word embedding of ith training target

In [152]:
i = 25
print(f'word embedding of "{training_data[i][1]}" = {np.dot(X_train[i:(i+1)], w) + b}')

word embedding of "Wonderful" = [[-0.82915357  0.45223223  0.69289802  1.51108107 -0.62852705  1.5968971
   1.02002046 -0.75671908 -0.66323505  1.12795119]]
